In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost.sklearn import XGBRegressor
from catboost import CatBoostRegressor
import lightgbm as lgb

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
def MAPE(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Data input

In [ ]:
# data
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

testing = test.copy()

target = "SalePrice"

df = pd.concat([train, test], axis = 0)
df.info()
df.head(5)

In [ ]:
for column in df.columns:
    try:
        df[column] = pd.to_numeric(df[column])
    except ValueError:
        pass

In [ ]:
df.drop(columns = ['Id'], inplace = True)

In [ ]:
plt.subplots(figsize = (10, 5))
sns.distplot(df[target].dropna(), kde = True, label = target, color = 'green', bins = 100)
plt.legend(prop = {'size': 12})
plt.show()

# Handling missing data

In [ ]:
plt.figure(figsize = (12, 10))
(df.loc[:, df.isnull().any()].isna().sum() / df.shape[0]).sort_values().plot(kind = 'barh', label = '% of missing values')
plt.axvline(x = 0.2, color = 'r', linestyle = '--', label = 'Reference line')
plt.legend()
plt.title('Percentage of missing values:')
plt.xlabel('% of missing data')
plt.show()

In [ ]:
# Manually dropping features which has more then 20% missing vlaues.
try:
    df.drop(columns = ['MiscFeature', 'Fence', 'PoolQC', 'FireplaceQu', 'Alley'], inplace = True)
except KeyError:
    print('Features already dropped')
    pass

columns_missing_less_200 = df[df.select_dtypes(exclude = object).columns].loc[:, df.isnull().sum() < 200].columns
for col in columns_missing_less_200:
    df[col] = df[col].fillna(df[col].median())
    
df[df.select_dtypes(exclude = object).columns].loc[:, df.isnull().any()].isna().sum()

In [ ]:
data_corr = df.corr().abs().unstack().sort_values(kind = "quicksort", ascending = False).reset_index()
data_corr.loc[data_corr['level_0'] == 'LotFrontage']

In [ ]:
# Filling missing values by most correlated feature's median
df['LotFrontage'] = df['LotFrontage'].fillna(df.groupby(['LotArea'])['LotFrontage'].transform('median'))
df['LotFrontage'] = df['LotFrontage'].fillna(df.groupby(['1stFlrSF'])['LotFrontage'].transform('median'))
df['LotFrontage'] = df['LotFrontage'].fillna(df.groupby(['MSSubClass'])['LotFrontage'].transform('median'))

df[df.select_dtypes(exclude = object).columns].loc[:, df.isnull().any()].isna().sum()

# Selecting numeric features

In [ ]:
corrMatrix = df.corr()
mask = np.zeros_like(corrMatrix)
mask[np.triu_indices_from(mask)] = True

plt.subplots(figsize=(28, 12))
sns.heatmap(corrMatrix, mask = mask, vmax = .3, square = False, annot = True)
plt.show()

In [ ]:
plt.subplots(figsize = (20, 5))
corrMatrix[target].drop([target]).sort_values().plot(kind = 'bar')
plt.axhline(y = 0.05, color = 'red', linestyle = '--')
plt.axhline(y = -0.05, color = 'red', linestyle = '--')
plt.show()

#Correlation with output variable
cor_target = abs(corrMatrix[target])

#Selecting correlated features
correlated_features = cor_target[cor_target >= 0.05]
print('Amount of total features: ', len(df.select_dtypes(include = object).columns))
print('Amount of correlated features: ', len(correlated_features))

In [ ]:
for i in range(0, df[correlated_features.index].shape[1], 6):
    sns.pairplot(data = df[correlated_features.index],
                x_vars =  df[correlated_features.index].columns[i:i+6],
                y_vars = ['SalePrice'], kind='reg')

## Removing some outliers

In [ ]:
df.loc[(df['BsmtFinSF1'] > 4000) & (df[target].notnull()), 'BsmtFinSF1'] = None
df.loc[(df['TotalBsmtSF'] > 4000) & (df[target].notnull()), 'TotalBsmtSF'] = None
df.loc[(df['GrLivArea'] > 4000) & (df[target] < 400000) & (df[target].notnull()), 'GrLivArea'] = None
df.loc[(df['1stFlrSF'] > 4000) & (df[target].notnull()), '1stFlrSF'] = None
df.loc[(df['LotFrontage'] > 200) & (df[target].notnull()), 'LotFrontage'] = None
df.loc[(df['LotArea'] > 60000) & (df[target].notnull()), 'LotArea'] = None
df.loc[(df['GarageArea'] > 1200) & (df[target] < 400000) & (df[target].notnull()), 'GarageArea'] = None


In [ ]:
for i in range(0, df[correlated_features.index].shape[1], 6):
    sns.pairplot(data = df[correlated_features.index],
                x_vars =  df[correlated_features.index].columns[i:i+6],
                y_vars = ['SalePrice'], kind='reg')

# Selecting categorical features

In [ ]:
# Check for any missing values in categorical features
df[df.select_dtypes(include = object).columns].loc[:, df.isnull().any()].isna().sum()

In [ ]:
for col in df.select_dtypes(include = object).columns:
    df[col] = df[col].fillna(df[col].mode()[0])
    
df[df.select_dtypes(include = object).columns].loc[:, df.isnull().any()].isna().sum()

In [ ]:
import scipy.stats as ss

def cramers_v(confusion_matrix):
    "calculate Cramers V statistic for categorial-categorial association"
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

cramers_values = []
obj_columns = df.select_dtypes(include = object).columns

for column in obj_columns:
    confusion_matrix = pd.crosstab(df[column], df[target])
    cramers_values.append(cramers_v(confusion_matrix.values))

cramers_values_df = pd.DataFrame()
cramers_values_df['feature'], cramers_values_df['value'] = obj_columns, cramers_values

plt.subplots(figsize = (12, 10))
plt.barh(cramers_values_df[cramers_values_df['value'] > 0].sort_values(['value'])['feature']
        ,cramers_values_df[cramers_values_df['value'] > 0].sort_values(['value'])['value'])
plt.axvline(x = 0.15, color = 'red', linestyle = '--', label = 'Reference line')
plt.title("Categorical feature correlation with the target")
plt.show()

cramers_values_df = cramers_values_df[cramers_values_df['value'] > 0.15]['feature'].to_list() # Select correlated features
len(cramers_values_df)

In [ ]:
# Final dataset
df = pd.concat([df[cramers_values_df], df[correlated_features.index]], axis = 1)
df

# Preparing data for modeling

In [ ]:
# Preparing features for analysis
dummy_features = df.select_dtypes(include = object).columns
    
df = pd.concat([df, pd.get_dummies(df[dummy_features], drop_first = True)], axis = 1, sort = False)
df.drop(columns = df[dummy_features], inplace = True)

df.tail()

In [ ]:
test = df[df[target].isnull()].drop([target], axis = 1).copy()
train = df[df[target].notnull()].copy()

train.head(5)

In [ ]:
# Drop NaN just to be safe
train.dropna(inplace = True)
test.dropna(inplace = True)

In [ ]:
# Separating target column from other features
y = train[target]
x = train.drop(columns = target)

# Train and Test dataset split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 42, shuffle = True)
x_train.shape, x_test.shape

# Selecting model

In [ ]:
CBR = CatBoostRegressor()
CBR_params = {
              'n_estimators':[n for n in range(600, 1400, 200)]
              ,'max_depth': [n for n in range(2, 10, 2)]
              ,'random_state' : [42]
              ,"learning_rate": [0.05, 0.03]
                }

CBR_model = GridSearchCV(CBR, param_grid = CBR_params, cv = 5, n_jobs = -1)
CBR_model.fit(x_train, y_train, silent = True)

CBR_predictions = CBR_model.predict(x_test)

print("R2 score: " + '%.3f' % r2_score(y_test, CBR_predictions))
print("RMSE score: ", f"{np.sqrt(mean_squared_error(y_test, CBR_predictions)):,.0f}")
print("MAE score: ", f"{mean_absolute_error(y_test, CBR_predictions):,.0f}")

print("Best Hyper Parameters: ",CBR_model.best_params_)
print("Best Score: " + '%.3f' % CBR_model.best_score_)

In [ ]:
# Plot feature importance
feature_importance = pd.Series(CBR_model.best_estimator_.feature_importances_, index = x_train.columns)

plt.figure(figsize = (10,8))
feature_importance.nlargest(15).sort_values(ascending = True).plot(kind = 'barh')
plt.show()

# Exporting predictions

In [ ]:
submission = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')

predictions = CBR_model.predict(test[x_train.columns])

submission["SalePrice"] = predictions
submission.to_csv("submission.csv", index = False)
print("Done")